In [1]:
import torch
from sklearn.datasets import fetch_20newsgroups
%load_ext autoreload
%autoreload 2

In [2]:
data = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))


In [3]:
N = 20
X = data['data'][:N]
y = data['target'][:N]
#y = [data['target_names'][y_] for y_ in data['target'][:N]]
from collections import Counter
Counter(y)

Counter({17: 3,
         0: 1,
         11: 3,
         10: 2,
         15: 2,
         4: 2,
         13: 2,
         12: 2,
         1: 1,
         6: 1,
         19: 1})

In [4]:
import re
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

#stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        #tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text
X = [preprocess_text(x) for x in X]

[nltk_data] Downloading package punkt to /home/kbogas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kbogas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kbogas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# import gensim
# wvmodel = gensim.models.KeyedVectors.load_word2vec_format('/home/kbogas/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [6]:
## import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')

In [7]:
# ft = fasttext.load_model('cc.en.300.bin')

In [8]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def dummy(doc):
    return doc
cv = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
    )  

doc_tokens = [x.split(" ") for x in X]
doc2count = cv.fit_transform(doc_tokens)



In [9]:
initial_embeddings = doc2count.copy()

In [10]:
from sklearn.model_selection import train_test_split
import numpy as np

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def load_data(adj, initial_embeddings, labels, eval_size=0.1, test_size=0.1, random_state=42):
    idx_train, idx_test = train_test_split(range(len(labels)), test_size=eval_size + test_size, random_state=random_state)
    idx_val, idx_test = train_test_split(range(len(labels)), test_size=test_size, random_state=random_state)
    
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    features = sparse_mx_to_torch_sparse_tensor(initial_embeddings)
    
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    
    return adj, features.t(), labels, idx_train, idx_val, idx_test

adj, features, labels, idx_train, idx_val, idx_test = load_data(doc2count, initial_embeddings, y, random_state=42)

In [11]:
type(features), features.shape

(torch.Tensor, torch.Size([2592, 20]))

In [12]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import accuracy
from pygcn.models import GCN
from pygcn.layers import GraphConvolution




cuda = False
nhid = 10
dropout = 0.1
lr =0.0001
weight_decay = 0.1
epochs= 10

np.random.seed(42)
torch.manual_seed(42)
if cuda:
    torch.cuda.manual_seed(seed)


adj, features, labels, idx_train, idx_val, idx_test = load_data(doc2count, initial_embeddings, y, random_state=42)

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=10,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()


Input <class 'torch.Tensor'> torch.Size([2592, 20])
Adj <class 'torch.Tensor'> torch.Size([20, 2592])


RuntimeError: sparse tensors do not have strides